# How to link Documents on common keywords (using Keybert)

A graph vector store retriever is a retriever that uses a graph vector store to retrieve documents. It is similar to a vector store retriever, except that it uses both vector similarity and graph connections to retriever documents
It uses the search methods implemented by a graph vector store, like similarity search and MMR, to query the texts in the graph vector store.

In this guide we will cover:

1. How to instantiate a retriever from a graph vectorstore;
2. How to specify the search type for the retriever;
3. How to specify additional search parameters, such as threshold scores and top-k.

## Creating a retriever from a graph vectorstore

You can build a retriever from a vectorstore using its [.as_retriever](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.graph_vectorstores.GraphVectorStore.html#langchain_core.vectorstores.VectorStore.as_retriever) method. Let's walk through an example.

First we instantiate a graph vectorstore. We will use a store backed by Cassandra [CassandraGraphVectorStore](https://api.python.langchain.com/en/latest/graph_vectorstores/langchain_community.graph_vectorstores.cassandra.CassandraGraphVectorStore.html) graph vectorstore:

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.graph_vectorstores import CassandraGraphVectorStore
from langchain_community.graph_vectorstores.extractors import KeybertLinkExtractor

from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("state_of_the_union.txt")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
keyword_extractor = KeybertLinkExtractor()
keyword_extractor.extract_many(texts)
embeddings = OpenAIEmbeddings()
graph_vectorstore = CassandraGraphVectorStore.from_documents(texts, embeddings)

We can then instantiate a retriever:

In [2]:
retriever = graph_vectorstore.as_retriever()

This creates a retriever (specifically a [GraphVectorStoreRetriever](https://api.python.langchain.com/en/latest/graph_vectorstores/langchain_core.graph_vectorstores.base.GraphVectorStoreRetriever.html)), which we can use in the usual way:

In [3]:
docs = retriever.invoke("what did the president say about ketanji brown jackson?")